# ERA5 temperature anomaly on pressure levels

## Import libraries

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download

plt.style.use("seaborn-v0_8-notebook")

## Select time range

In [ ]:
start = "2022-01"
stop = None

## Define request

In [ ]:
collection_id = "reanalysis-era5-pressure-levels-monthly-means"
request = {
    "format": "grib",
    "product_type": "monthly_averaged_reanalysis",
    "variable": "temperature",
    "pressure_level": [
        "1",
        "2",
        "3",
        "5",
        "7",
        "10",
        "20",
        "30",
        "50",
        "70",
        "100",
        "125",
        "150",
        "175",
        "200",
        "225",
        "250",
        "300",
        "350",
        "400",
        "450",
        "500",
        "550",
        "600",
        "650",
        "700",
        "750",
        "775",
        "800",
        "825",
        "850",
        "875",
        "900",
        "925",
        "950",
        "975",
        "1000",
    ],
    "time": "00:00",
}
requests = download.update_request_date(request, start=start, stop=stop)

## Define function

In [ ]:
def zonal_mean(ds):
    return ds.mean("longitude", keep_attrs=True)

## Download data

In [ ]:
ds_zonal = download.download_and_transform(
    collection_id,
    requests,
    transform_func=zonal_mean,
    chunks={"year": 1},
)

## Compute anomalies

In [ ]:
ds_mean = diagnostics.spatial_weighted_mean(ds_zonal, dim=("latitude"))
ds_grouped = ds_mean.groupby("forecast_reference_time.month")
with xr.set_options(keep_attrs=True):
    ds_anoma = ds_grouped - ds_grouped.mean("forecast_reference_time")
ds_anoma["t"].attrs["long_name"] = ds_mean["t"].attrs["long_name"] + " Anomaly"

## Hovmöller Diagram

In [ ]:
ds_anoma["t"].plot.contourf(
    x="time",
    yscale="log",
    robust=True,
    levels=15,
    yincrease=False,
    figsize=(15, 5),
    cbar_kwargs={"orientation": "horizontal"},
)
_ = plt.title("ERA5 temperature on pressure levels")